In [1]:
import os
from openai import OpenAI
import sys
import config
import getpass
import re

In [2]:
from langchain.document_loaders import WikipediaLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader
from langchain.vectorstores.utils import filter_complex_metadata
from PyPDF2 import PdfReader

In [3]:
os.environ["OPENAI_API_KEY"] = config.OPENAI_API_KEY

In [4]:
pdfpath="Resources/Manual_Usuario.pdf"
loader = PyPDFLoader(pdfpath)
docs = loader.load()

In [5]:
def get_pdf_text(pdf_docs):
    text=""
    #for pdf in pdf_docs:
    pdf_reader= PdfReader(pdf_docs)
    for page in pdf_reader.pages:
        text+= page.extract_text()
    return  text

In [6]:
docs=get_pdf_text(pdfpath)

In [7]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=500)
data = text_splitter.split_text(docs)

In [8]:
from langchain.vectorstores import Chroma
#from langchain.embeddings import OpenAIEmbeddings
from langchain_openai import OpenAIEmbeddings

In [9]:
embeddings = OpenAIEmbeddings()

In [10]:
store = Chroma.from_texts(
    data, 
    embeddings, 
    #ids = [f"{item.metadata['source']}-{index}" for index, item in enumerate(data)],
    collection_name="Inrobicsv2", 
persist_directory='db',
)
store.persist()

In [11]:
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
#from langchain.chat_models import ChatOpenAI
from langchain_openai import ChatOpenAI

In [12]:
template = """You are a bot that answers questions about the context provided.
If you don't know the answer, simply state that you don't know.

{context}

Question: {question}"""

PROMPT = PromptTemplate(
    template=template, input_variables=["context", "question"]
)

In [13]:
llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo")

In [14]:
qa_with_source = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=store.as_retriever(),
    chain_type_kwargs={"prompt": PROMPT, },
    return_source_documents=True,
)

In [15]:
qa_with_source.invoke("Qué es Inrobics?")

{'query': 'Qué es Inrobics?',
 'result': 'Inrobics es un producto sanitario certificado de clase I en la Unión Europea, desarrollado por Inrobics Social Robotics, S.L.L. Se trata de un sistema de rehabilitación que ofrece actividades terapéuticas configurables y personalizables para pacientes, con diferentes niveles de dificultad.',
 'source_documents': [Document(page_content='Inrobics  es un producto sanitario \ncertificado de clase l en la Unión \nEuropea  \n Inrobics Social Robotics, S.L.L.  \nAv. Gregorio Peces Barba 1,  \n28919 Leganés (Madrid) - SPAIN  \n Por favor, lea los manuales \nproporcionados  \n Número de serie: 6 \n (01)08437025423002  \n(11)23 1218  \n(21)6 \n \n  \nWelcome to the next revolution in Rehab  \n \n3 \nContenido  \n \n1. Introducción  ................................ ................................ .................  4 \n1.1. Convenciones  ................................ ................................ .............  5 \n1.2. Previsión de uso  ..........

In [16]:
qa_with_source.invoke("Qué actividades ofrece Inrobics?")

{'query': 'Qué actividades ofrece Inrobics?',
 'result': 'Inrobics ofrece 12 actividades diferentes, que incluyen EVAL, WARM UP, DYNAMIC, AVDS, SYMBOLIC, DANCE, STATUES, ROBIC SAYS, y ROBIC LIES, entre otras.',
 'source_documents': [Document(page_content='diferentes niveles  de dificultad. Estas actividades  terapéuticas son fácilmente \nconfigurables  y personalizables para que el profesional pueda individualizar  el \ncaso de uso a cada paciente.  \nUna sesión de Inrobics se compone de bloques de trabajo formados por los \nexergames con descansos activos  intercalados  (en caso de que se prescriban). \nEl tiempo de duración de una sesión con Inrobics es completamente \nindividualizable a las necesidades del usuario, pudiendo ir desde 10 hasta 25 \nminutos aproximadamente , o lo considerado por el profesional responsable.   \nExergames  \nActualmente la plataforma cuenta con 12 actividades . Sus características son \nlas siguientes:  \n● EVAL: Actividad para monitorizar el rango  de m

In [17]:
qa_with_source.invoke("Dame una lista de las  actividades que ofrece Inrobics, solo los nombres")

{'query': 'Dame una lista de las  actividades que ofrece Inrobics, solo los nombres',
 'result': "- EVAL\n- WARM UP\n- DYNAMIC\n- AVDS\n- SYMBOLIC\n- DANCE\n- STATUES\n- ROBIC SAYS\n- ROBIC LIES\n- DON'T STOP\n- COLORS",
 'source_documents': [Document(page_content='minutos aproximadamente , o lo considerado por el profesional responsable.   \nExergames  \nActualmente la plataforma cuenta con 12 actividades . Sus características son \nlas siguientes:  \n● EVAL: Actividad para monitorizar el rango  de movimiento  articular  de \nmiembros superiores del usuario al realizar diferentes movimientos que \nle propone Robic.  \n● WARM UP: El robot  Robic propone unas secuencias de movimientos que \nrealiza con la persona de manera simultánea. Puede utilizarse  como un \ncalentamiento previo al resto de la sesión.  \n● DYNAMIC: Robic propone secuencias de  ejercicios físicos  dirigidas al \nentrenamiento  del rango de movimiento activo y aspectos funcionales de \nmovilidad . Se trabaja  mediante